In [1]:
import json
import os
import glob

In [2]:
negatives = glob.glob('/home/husein/Malaya-Dataset/fake-news/negative/*.json')
positives = glob.glob('/home/husein/Malaya-Dataset/fake-news/positive/*.json')
negatives

['/home/husein/Malaya-Dataset/fake-news/negative/clean-gossipcop_real2.json',
 '/home/husein/Malaya-Dataset/fake-news/negative/clean-gossipcop_real3.json',
 '/home/husein/Malaya-Dataset/fake-news/negative/1.json',
 '/home/husein/Malaya-Dataset/fake-news/negative/clean-gossipcop_real1.json',
 '/home/husein/Malaya-Dataset/fake-news/negative/clean-politifact_real.json',
 '/home/husein/Malaya-Dataset/fake-news/negative/2.json']

In [3]:
texts, labels = [], []
for negative in negatives:
    with open(negative) as fopen:
        x = json.load(fopen)
    texts.extend(x)
    labels.extend([1] * len(x))

In [4]:
for positive in positives:
    with open(positive) as fopen:
        x = json.load(fopen)
    texts.extend(x)
    labels.extend([0] * len(x))

In [5]:
import numpy as np
import pandas as pd

In [6]:
np.unique(labels, return_counts = True)

(array([0, 1]), array([15191, 26832]))

In [7]:
df = pd.read_csv('Malaya-Dataset/fake-news/malaysia-scraping-syazanihussin.csv').dropna()
df.head()

,News,Label
0,"Produk Kod E Mengandungi Lemak Babi. Awas, jan...",Fake
1,Jabatan Kemajuan Islam Malaysia memperjelaskan...,Real
2,Roti Massimo Mengandungi DNA Babi. Roti produk...,Fake
3,Jabatan Kemajuan Islam Malaysia (JAKIM) melalu...,Real
4,Tiada Pembayaran Pencen Selepas Persaraan 2021...,Fake


In [8]:
df_fake = df.loc[df['Label'] == 'Fake']
df_real = df.loc[df['Label'] == 'Real']
df_fake.shape, df_real.shape

((239, 2), (232, 2))

In [9]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_Y, test_Y = train_test_split(texts, labels, test_size = 0.2)
len(train_X), len(test_X)

(33618, 8405)

In [10]:
train_X.extend(df_fake['News'].tolist())
train_Y.extend([0] * len(df_fake['News'].tolist()))

train_X.extend(df_real['News'].tolist())
train_Y.extend([1] * len(df_real['News'].tolist()))

In [11]:
len(train_X), len(train_Y)

(34089, 34089)

In [12]:
from sklearn.utils import shuffle

train_X, train_Y = shuffle(train_X, train_Y)

In [13]:
with open('Malaya-Dataset/synonym/synonym0.json') as fopen:
    s = json.load(fopen)
    
with open('Malaya-Dataset/synonym/synonym1.json') as fopen:
    s1 = json.load(fopen)

In [14]:
synonyms = {}
for l, r in (s + s1):
    if l not in synonyms:
        synonyms[l] = r + [l]
    else:
        synonyms[l].extend(r)
synonyms = {k: list(set(v)) for k, v in synonyms.items()}

In [15]:
import random

def augmentation(s, maximum = 1.0):
    s = s.split()
    for i in range(int(len(s) * maximum)):
        index = random.randint(0, len(s) - 1)
        word = s[index]
        sy = synonyms.get(word, [word])
        sy = random.choice(sy)
        s[index] = sy
    return s

In [16]:
aug = [' '.join(augmentation(x[0])) for _ in range(3)]
aug = list(set(aug))
x[0], aug

('mencetak seorang wanita iranian telah dijatuhi hukuman enam tahun penjara selepas pengawal revolusioner iran mencari rumahnya dan mendapati sebuah buku catatan yang mengandungi kisah fiksyen yang ditulisnya tentang seorang wanita yang dilontar hingga mati, menurut kajian eurasia. Golrokh ebrahimi iraee, 35, adalah isteri banduan politik sadashhi sadeghi, 36, yang menjalani hukuman penjara 19 tahun kerana menjadi aktivis hak asasi manusia. "apabila unit perisikan pengawal revolusioner datang untuk menangkap suaminya, mereka menyerbu apartmen mereka tanpa waran dan mendapati draf cerita yang ditulis oleh ebrahimi iraee," artikel itu dinyatakan. "salah satu draf yang dirampas adalah cerita mengenai wanita melontar hingga mati kerana perzinaan yang tidak pernah diterbitkan, tidak pernah dikemukakan kepada sesiapa pun," artikel itu dinyatakan. "cerita itu menyusuli kisah seorang protagonis yang menonton filem mengenai pembasmian wanita di bawah undang-undang islam kerana berzina.',
 ['men

In [17]:
from tqdm import tqdm

X, Y = [], []

for i in tqdm(range(len(train_X))):
    aug = [' '.join(augmentation(train_X[i])) for _ in range(3)]
    aug.append(train_X[i])
    aug = list(set(aug))
    X.extend(aug)
    Y.extend([train_Y[i]] * len(aug))

100%|██████████| 34089/34089 [01:45<00:00, 323.18it/s]


In [19]:
len(X), len(Y)

(135615, 135615)

In [20]:
X, Y = shuffle(X, Y)

In [21]:
import pickle

with open('relevant-dataset.pkl', 'wb') as fopen:
    pickle.dump({'train_X': X, 'train_Y': Y,
                'test_X': test_X, 'test_Y': test_Y}, fopen)